# Main Routine

This code is an updated python adaption of Yvan Dossman's original Matlab code
by Callum Shaw
13/12/2019


In [8]:
%matplotlib qt
#notebook backend important

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os.path
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askopenfilenames

In [77]:
def index_find(depths,click):
    lst = depths>click
    for i,v in enumerate(lst):
        if v==True:
            return i
        

Loading in the experiment density data from excel

In [9]:
excel_path = askopenfilename() #path to excel doc

exp_num = int(input('Experiment number:'))
data = pd.read_excel(excel_path, skiprows=3) #has 3 unwanted rows at top

#getting data from excel doc
rho_bottom = data.iloc[exp_num, 10]
rho_top = data.iloc[exp_num, 11]
depth = data.iloc[exp_num, 7]


Experiment number:35


Loading in the background image and choosing region want to analyse (should be homogeneous). This image should contain no topography

In [10]:
background_path = askopenfilename() 
b_image = cv2.imread(background_path,0)

In [11]:

plt.figure()
plt.title('Background Image')
plt.imshow(b_image, cmap='gist_gray')
print('Select the left then the right portion of the region you want to average over')
crop_region = plt.ginput(2) #select the left then the right 


Select the left then the right portion of the region you want to average over


Loading in the foreground images

In [12]:
#now loading in the foreground photos
foreground_path = askopenfilenames()
no_images = len(foreground_path)
os.mkdir('{}/results'.format(os.path.dirname(foreground_path[0]))) #making a directory to store results in same location as before


Now analysing the foreground photos

In [13]:
f_image = cv2.imread(foreground_path[0],0) #choosing the imag

f1=plt.figure()
plt.title('Foreground Image')
plt.imshow(f_image,  cmap='gist_gray') #plotting the f_image_crop check if happy 
print('Position of free surface then tank bottom')
free_surface = plt.ginput(2)

f2=plt.figure()
f_image_crop = f_image[int(free_surface[0][1]):int(free_surface[1][1]),int(crop_region[0][0]):int(crop_region[1][0])]
plt.title('Foreground Image Cropped')
plt.imshow(f_image_crop,  cmap='gist_gray') #plotting the f_image_crop check if happy 

#cropping the background image the same way
b_image_crop = b_image[int(free_surface[0][1]):int(free_surface[1][1]),int(crop_region[0][0]):int(crop_region[1][0])]
b_image_crop = b_image_crop*0+1 #in the case of an initially varing stratification

Position of free surface then tank bottom


Absorbtion Calculation

In [59]:
ratio =f_image_crop/b_image_crop
log_ratio = np.log(ratio)

#getting rid of unwated inf_values
log_ratio[np.isinf(log_ratio)]=np.nan

mean_ratio_profile = np.mean(log_ratio,1)
intensity = mean_ratio_profile[::-1]

depth_array = np.linspace(0,depth,len(mean_ratio_profile))
gamma=depth/(int(free_surface[1][1])-int(free_surface[0][1]))

plt.plot(intensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Raw Absorption Profile')

Text(0.5, 1.0, 'Raw Absorption Profile')

Spline of the intensity profile- fixing the top and bottom. Using constant at top and linear slope at bottom

In [143]:
#firstly looking at the top section 
splintensity=np.copy(intensity)

plt.plot(intensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Top Section of Absorption Profile')
print('Choose bottom of top section')
topsection = plt.ginput(1)[0][1]

ztop=index_find(depth_array,topsection) 
splintensity[ztop:]=splintensity[ztop]

Choose bottom of top section


In [144]:
#now looking at the bottom section 

plt.plot(splintensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')
plt.title('Select Bottom Section of Absorption Profile')
print('Choose top then bottom')
bottomsection = plt.ginput(2)


Choose top then bottom


In [149]:
slope = (bottomsection[0][0]-bottomsection[1][0])/bottomsection[0][1]
intercept = bottomsection[0][1]-bottomsection[0][0]/slope
zbot=index_find(depth_array, bottomsection[0][1]) 
splintensity[0:zbot]=slope*(np.linspace(0,bottomsection[0][1], zbot)-intercept)

plt.plot(splintensity,depth_array)
plt.xlabel('ln (Ix/Ix0)')
plt.ylabel('z (m)')

Check that plot above looks good or repeat as needed